# Example: Pause and Continue
This notebook shows how one can execute two functions for one experiment by setting the status to `paused`, and unpausing it later on with a different execution function. This example is heavily based on the `example_general_usage.ipynb` notebook.

To execute this notebook you need to install:
```
pip install py_experimenter
pip install scikit-learn
```

## Experiment Configuration File
First we define an experiment configuraiton file. Note that in comparison to the basic example two resultfields `paused_at_seconds` and `resumed_at_seconds` were added.

In [6]:
import os

content = """
[PY_EXPERIMENTER]
provider = sqlite
database = py_experimenter
table = example_pause_and_continue

keyfields = dataset, cross_validation_splits:int, seed:int, kernel
dataset = iris
cross_validation_splits = 5
seed = 2:6:2
kernel = linear, poly, rbf, sigmoid

resultfields = pipeline:LONGTEXT, train_f1:DECIMAL, train_accuracy:DECIMAL, test_f1:DECIMAL, test_accuracy:DECIMAL, paused_at_seconds:DOUBLE, resumed_at_seconds:DOUBLE
resultfields.timestamps = false

[CUSTOM]
path = sample_data

[codecarbon]
offline_mode = False
measure_power_secs = 25
tracking_mode = process
log_level = error
save_to_file = True
output_dir = output/CodeCarbon
"""
# Create config directory if it does not exist
if not os.path.exists('config'):
    os.mkdir('config')
    
# Create config file
experiment_configuration_file_path = os.path.join('config', 'example_general_usage.cfg')
with open(experiment_configuration_file_path, "w") as f: 
  f.write(content)

## Defining Pausing Execution Function
Next we fill the table, define the execution function that gets paused after five seconds and run this execution function.

In [7]:
import datetime

from py_experimenter.experimenter import ExperimentStatus, PyExperimenter
from py_experimenter.result_processor import ResultProcessor

experimenter = PyExperimenter(experiment_configuration_file_path=experiment_configuration_file_path, name='example_notebook')

experimenter.fill_table_from_config()

def pause_after_5_seconds(parameters: dict, result_processor: ResultProcessor, custom_config: dict):
    import time
    time.sleep(5)
    result_processor.process_results({
        'paused_at_seconds': datetime.datetime.now().timestamp()
    })
    return ExperimentStatus.PAUSED.value



experimenter.execute(pause_after_5_seconds, max_experiments=1)

2023-09-25 14:15:26,226  | py-experimenter - INFO     | Initialized and connected to database
2023-09-25 14:15:26,286  | py-experimenter - INFO     | 12 rows successfully added to database. 0 rows were skipped.


2023-09-25 14:15:37,473  | py-experimenter - INFO     | All configured executions finished.


## Showcase Paused Execution
Below we only show that the execution of the experiment with `id=1` has been paused.

In [8]:
experimenter.get_table()

,ID,dataset,cross_validation_splits,seed,kernel,creation_date,status,start_date,name,machine,pipeline,train_f1,train_accuracy,test_f1,test_accuracy,paused_at_seconds,resumed_at_seconds,end_date,error
0,1,iris,5,2,linear,2023-09-25 14:15:26,paused,2023-09-25 14:15:26,example_notebook,Worklaptop,None,None,None,None,None,1.695644e+09,None,2023-09-25 14:15:37,None
1,2,iris,5,4,linear,2023-09-25 14:15:26,created,None,None,None,None,None,None,None,None,NaN,None,None,None
2,3,iris,5,6,linear,2023-09-25 14:15:26,created,None,None,None,None,None,None,None,None,NaN,None,None,None
3,4,iris,5,2,poly,2023-09-25 14:15:26,created,None,None,None,None,None,None,None,None,NaN,None,None,None
4,5,iris,5,4,poly,2023-09-25 14:15:26,created,None,None,None,None,None,None,None,None,NaN,None,None,None
5,6,iris,5,6,poly,2023-09-25 14:15:26,created,None,None,None,None,None,None,None,None,NaN,None,None,None
6,7,iris,5,2,rbf,2023-09-25 14:15:26,created,None,None,None,None,None,None,None,None,NaN,None,None,None
7,8,iris,5,4,rbf,2023-09-25 14:15:26,created,None,None,None,None,None,None,None,None,NaN,None,None,None
8,9,iris,5,6,rbf,2023-09-25 14:15:26,created,None,None,None,None,None,None,None,None,NaN,None,None,None
9,10,iris,5,2,sigmoid,2023-09-25 14:15:26,created,None,None,None,None,None,None,None,None,NaN,None,None,None


## Define resuming execution function

Lastly, we can define a second execution function that resumes the paused execution function. After running this execution function, the experiment is finished and the resulting table is shown.

In [9]:
import random

import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import cross_validate
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

def resume_after_5_seconds(parameters: dict, result_processor: ResultProcessor, custom_config: dict):
    result_processor.process_results({
        'resumed_at_seconds': datetime.datetime.now().timestamp()
    })
    seed = parameters['seed']
    random.seed(seed)
    np.random.seed(seed)

    data = load_iris()
    # In case you want to load a file from a path
    # path = os.path.join(custom_config['path'], parameters['dataset'])
    # data = pd.read_csv(path)

    X = data.data
    y = data.target

    model = make_pipeline(StandardScaler(), SVC(kernel=parameters['kernel'], gamma='auto'))
    result_processor.process_results({
        'pipeline': str(model)
    })

    if parameters['dataset'] != 'iris':
        raise ValueError("Example error")

    scores = cross_validate(model, X, y,
                            cv=parameters['cross_validation_splits'],
                            scoring=('accuracy', 'f1_micro'),
                            return_train_score=True
                            )

    result_processor.process_results({
        'train_f1': np.mean(scores['train_f1_micro']),
        'train_accuracy': np.mean(scores['train_accuracy'])
    })

    result_processor.process_results({
        'test_f1': np.mean(scores['test_f1_micro']),
        'test_accuracy': np.mean(scores['test_accuracy'])
    })


experimenter.unpause_experiment(1, resume_after_5_seconds)

experimenter.get_table()

,ID,dataset,cross_validation_splits,seed,kernel,creation_date,status,start_date,name,machine,pipeline,train_f1,train_accuracy,test_f1,test_accuracy,paused_at_seconds,resumed_at_seconds,end_date,error
0,1,iris,5,2,linear,2023-09-25 14:15:26,done,2023-09-25 14:15:26,example_notebook,Worklaptop,"Pipeline(steps=[('standardscaler', StandardSca...",0.971667,0.971667,0.966667,0.966667,1.695644e+09,1.695644e+09,2023-09-25 14:15:44,None
1,2,iris,5,4,linear,2023-09-25 14:15:26,created,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,None,None
2,3,iris,5,6,linear,2023-09-25 14:15:26,created,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,None,None
3,4,iris,5,2,poly,2023-09-25 14:15:26,created,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,None,None
4,5,iris,5,4,poly,2023-09-25 14:15:26,created,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,None,None
5,6,iris,5,6,poly,2023-09-25 14:15:26,created,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,None,None
6,7,iris,5,2,rbf,2023-09-25 14:15:26,created,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,None,None
7,8,iris,5,4,rbf,2023-09-25 14:15:26,created,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,None,None
8,9,iris,5,6,rbf,2023-09-25 14:15:26,created,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,None,None
9,10,iris,5,2,sigmoid,2023-09-25 14:15:26,created,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,None,None


## CodeCarbon Entries
Note that for each execution a different `CodeCarbon` entry is created.

In [10]:
experimenter.get_codecarbon_table()

,ID,experiment_id,codecarbon_timestamp,project_name,run_id,duration_seconds,emissions_kg,emissions_rate_kg_sec,cpu_power_watt,gpu_power_watt,...,cpu_model,gpu_count,gpu_model,longitude,latitude,ram_total_size,tracking_mode,on_cloud,power_usage_efficiency,offline_mode
0,1,1,2023-09-25T14:15:37,codecarbon,1c8540a9-1cf6-44b3-b8d1-6c521672e96a,5.050751,2.112876e-05,0.000004,42.5,0.0,...,12th Gen Intel(R) Core(TM) i7-1260P,None,None,9.7183,52.3709,15.474876,process,N,1.0,0
1,2,1,2023-09-25T14:15:44,codecarbon,124b8fff-3efd-4f43-9327-c4c1b48126cd,0.132071,5.167705e-07,0.000004,42.5,0.0,...,12th Gen Intel(R) Core(TM) i7-1260P,None,None,9.7183,52.3709,15.474876,process,N,1.0,0
